In [1]:
import pandas as pd
from pathlib import Path


raw_data_path = Path("../data/raw")
preprocessed_file = Path("../data/preprocessed/text-and-image.csv")

In [2]:
if not preprocessed_file.exists():
    main_df = pd.DataFrame(columns=["text", "image_name", "blocked"])
else: 
    main_df = pd.read_csv(preprocessed_file)

In [10]:
def display_label_stats(df) -> None:
    display(df.loc[:, df.columns == "blocked"].value_counts().reset_index(name="count"))

## neuralcatcher/hateful_memes
Все изображения загружаются локально из-за ограничения rate limit

In [11]:
from vk_mod.data.download import load_from_huggingface
load_from_huggingface("neuralcatcher/hateful_memes")
temp_df = pd.read_csv(Path(raw_data_path, "hateful_memes.csv"))
temp_df.head(5)

Repo card metadata block was not found. Setting CardData to empty.


,Unnamed: 0,id,img,label,text
0,0,42953,img/42953.png,0,its their character not their color that matters
1,1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...
2,2,13894,img/13894.png,0,putting bows on your pet
3,3,37408,img/37408.png,0,i love everything and everybody! except for sq...
4,4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h..."


In [ ]:
import argostranslate
import argostranslate.package

def install_translation_model():
    argostranslate.package.update_package_index()
    available_packages = argostranslate.package.get_available_packages()
    package_to_install = next(
        filter(
            lambda x: x.from_code == "en" and x.to_code == "ru",
            available_packages
        )
    )
    argostranslate.package.install_from_path(package_to_install.download())

install_translation_model()

In [13]:
import argostranslate.translate
import pandas as pd
from tqdm import tqdm


FROM_LANG = "en"
TO_LANG = "ru"

processed_data = []
for index, row in tqdm(temp_df.iterrows(), total=len(temp_df)):
    filename = row['img'].split('/')[-1]
    try:
        text_ru = argostranslate.translate.translate(
            row['text'], 
            FROM_LANG, 
            TO_LANG
        )
        processed_data.append({
            'text': text_ru,
            'image_name': filename,
            'blocked': row['label']
        })
    except Exception as e:
        print(f"Ошибка перевода для строки {index}: {e}")

temp_df = pd.DataFrame(processed_data)
display(temp_df.sample(5))
display_label_stats(temp_df)

100%|██████████| 8500/8500 [53:08<00:00,  2.67it/s]  


,text,image_name,blocked
4057,если он женится на мне тогда почему он трахает...,81409.png,0
6852,"Познакомься с Джоном Дикарем, играющим главную...",15964.png,1
7609,"Ничего не делать с глобальной бедностью, сидя ...",53810.png,0
7416,Новый дизайн ремней безопасности: на 45% меньш...,84069.png,1
1612,"Что делать, если правительство финансирует усы...",84209.png,0


,blocked,count
0,0,5481
1,1,3019


In [ ]:
main_df = pd.concat([main_df, temp_df])
display(main_df)
display_label_stats(main_df)

,text,image_name,blocked
0,"Его характер, а не цвет имеет значение",42953.png,0
1,"Не бойтесь снова любить, все не такие, как ваш...",23058.png,0
2,Наносить луки на вашего питомца,13894.png,0
3,"Я люблю все и всех! За исключением белок, я не...",37408.png,0
4,"Все любят шоколадное печенье, даже Гитлер.",82403.png,0
...,...,...,...
8495,Никто не хочет меня повесить.,10423.png,1
8496,Когда Бог дарует вам ребенка после 20 лет брак...,98203.png,1
8497,Социальные сети: равенство! Позитивность тела!...,36947.png,1
8498,Плохой день? Ты можешь быть сиамским близнецом...,16492.png,1


,blocked,count
0,0,5481
1,1,3019


## victorcallejasf/multimodal-hate-speech

In [ ]:
from pathlib import Path
dataset_path = Path("../data/MMHS/MMHS150K_GT.json")
dataset_images = Path("../data/MMHS/img_resized")

In [ ]:
import json
with open(dataset_path, 'r', encoding='utf-8') as f:
    data_dict = json.load(f)

In [ ]:
import argostranslate.translate
from tqdm import tqdm
from itertools import islice

max_items = 50000
procesesed_data = []

for index, row in tqdm(islice(data_dict.items(), max_items), total=max_items):
    text = argostranslate.translate.translate(row['tweet_text'], "en", "ru")
    labels = [1 for label in row['labels'] if label != 0]
    label = 1 if sum(labels) >= 2 else 0
    filename = f"{index}.jpg"
    image_path = Path(dataset_images, filename)
    if (not image_path.exists()):
        continue
    image_path.rename(Path(images_dir, filename))
    procesesed_data.append({
        "image_name": filename,
        "text": text,
        "blocked": label
    })
temp_df = pd.DataFrame(procesesed_data)
display(temp_df.sample(5))
display(temp_df.loc[:, temp_df.columns == "blocked"].value_counts().reset_index(name="count"))

c:\Users\ShapeCat\source\repos\ToxicPostDetection\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 50000/50000 [9:59:11<00:00,  1.39it/s]   


,image_path,text,blocked
30729,1037421263600406528.jpg,""" Не прикасайтесь ко мне, — говорит сенатор Ру...",0
13652,1106688752523640834.jpg,"Я думаю, мы называем это Redneck изобретательн...",1
4123,1109886472251953152.jpg,"Все эти плохие сучки обожают Оставил ее нигга,...",0
10964,1113121489145004032.jpg,@cumeconomics Нигга Iint reading a https://t....,0
2133,1114657282271395840.jpg,"Получить нигга, как я https://t.co/Kndu7bsgd1",0


,blocked,count
0,0,36928
1,1,13072


In [ ]:
import argostranslate.translate
from tqdm import tqdm
from itertools import islice

max_items = 50000
procesesed_data = []

for index, row in tqdm(islice(data_dict.items(), max_items), total=max_items):
    text = argostranslate.translate.translate(row['tweet_text'], "en", "ru")
    labels = [1 for label in row['labels'] if label != 0]
    label = 1 if sum(labels) >= 2 else 0
    filename = f"{index}.jpg"
    image_path = Path(dataset_images, filename)
    if (not image_path.exists()):
        continue
    image_path.rename(Path(images_dir, filename))
    procesesed_data.append({
        "image_name": filename,
        "text": text,
        "blocked": label
    })
temp_df = pd.DataFrame(procesesed_data)
display(temp_df.sample(5))
display(temp_df.loc[:, temp_df.columns == "blocked"].value_counts().reset_index(name="count"))

c:\Users\ShapeCat\source\repos\ToxicPostDetection\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 50000/50000 [9:59:11<00:00,  1.39it/s]   


,image_path,text,blocked
30729,1037421263600406528.jpg,""" Не прикасайтесь ко мне, — говорит сенатор Ру...",0
13652,1106688752523640834.jpg,"Я думаю, мы называем это Redneck изобретательн...",1
4123,1109886472251953152.jpg,"Все эти плохие сучки обожают Оставил ее нигга,...",0
10964,1113121489145004032.jpg,@cumeconomics Нигга Iint reading a https://t....,0
2133,1114657282271395840.jpg,"Получить нигга, как я https://t.co/Kndu7bsgd1",0


,blocked,count
0,0,36928
1,1,13072


In [ ]:
main_df = pd.concat([main_df, temp_df])
display(main_df)
display(main_df.loc[:, main_df.columns == "blocked"].value_counts().reset_index(name="count"))

,Unnamed: 0,text,image_name,blocked
0,0,"Его характер, а не цвет имеет значение",42953.png,0
1,1,"Не бойтесь снова любить, все не такие, как ваш...",23058.png,0
2,2,Наносить луки на вашего питомца,13894.png,0
3,3,"Я люблю все и всех! За исключением белок, я не...",37408.png,0
4,4,"Все любят шоколадное печенье, даже Гитлер.",82403.png,0
...,...,...,...,...
58495,49995,"@TopRoman_ Nigga - это те провода, которые я в...",1114763298015080448.jpg,0
58496,49996,Предвкушение... Желание. Накопленное разочаров...,1105169262640066562.jpg,0
58497,49997,Полная галерея: https://t.co/9x42BlpFva Ариэлл...,1114979009882918912.jpg,0
58498,49998,Ван Дайк появится на слушаниях по облигациям в...,1035886489752625152.jpg,0


,blocked,count
0,0,42409
1,1,16091


# Final

## Clean

In [15]:
main_df = (main_df
           .drop("Unnamed: 0", axis=1, errors="ignore")           
           .drop_duplicates()
           .dropna(subset=["text", "image_name", "blocked"]))
main_df['blocked'] = main_df['blocked'].astype(int)

In [16]:
display_label_stats(main_df)
print(main_df.shape)

,blocked,count
0,0,5481
1,1,3019


(8500, 3)


## Save

In [17]:
preprocessed_file.parent.mkdir(exist_ok=True)
main_df.to_csv(preprocessed_file, index=False)